### Вебинар 6. Двухуровневые модели рекомендаций

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Для категориальных переменных
from sklearn.preprocessing import LabelEncoder

# Матричная факторизация
from implicit import als

# Модель второго уровня
from lightgbm import LGBMClassifier

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# Написанные нами функции
from metrics import precision_at_k, recall_at_k
from utils import prefilter_items
from recommenders import MainRecommender

In [3]:
data = pd.read_csv('retail_train.csv')
item_features = pd.read_csv('product.csv')
user_features = pd.read_csv('hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)


# Важна схема обучения и валидации!
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 
# подобрать размер 2-ого датасета (6 недель) --> learning curve (зависимость метрики recall@k от размера датасета)
val_lvl_1_size_weeks = 6
val_lvl_2_size_weeks = 3

data_train_lvl_1 = data[data['week_no'] < data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]
data_val_lvl_1 = data[(data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) &
                      (data['week_no'] < data['week_no'].max() - (val_lvl_2_size_weeks))]

data_train_lvl_2 = data_val_lvl_1.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться
data_val_lvl_2 = data[data['week_no'] >= data['week_no'].max() - val_lvl_2_size_weeks]

data_train_lvl_1.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [ ]:
n_items_before = data_train_lvl_1['item_id'].nunique()

data_train_lvl_1 = prefilter_items(data_train_lvl_1, take_n_popular=5000)

n_items_after = data_train_lvl_1['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

In [ ]:
recommender = MainRecommender(data_train_lvl_1)

Задание 1.
A) Попробуйте различные варианты генерации кандидатов. Какие из них дают наибольший recall@k ?

Пока пробуем отобрать 50 кандидатов (k=50)

Качество измеряем на data_val_matcher: следующие 6 недель после трейна

Дают ли own recommendtions + top-popular лучший recall?

B)* Как зависит recall@k от k? Постройте для одной схемы генерации кандидатов эту зависимость для k = {20, 50, 100, 200, 500}

C)* Исходя из прошлого вопроса, как вы думаете, какое значение k является наиболее разумным?

In [ ]:
result_lvl_1 = data_val_lvl_1.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_1.columns=['user_id', 'actual']

In [ ]:
result_lvl_1.head(2)

In [ ]:
result_lvl_1['user_id']

In [ ]:
result_lvl_1['als_rec_50'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_als_recommendations(x, N=50))
result_lvl_1['own_rec_50'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=50))
result_lvl_1['sim_items_rec_50'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_similar_items_recommendation(x, N=50))
result_lvl_1['sim_user_rec_50'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_similar_users_recommendation(x, N=50))

result_lvl_1.head(2)

In [ ]:
recall_total = pd.DataFrame()
recall_total['user_id'] = result_lvl_1['user_id']
recall_total['als_rec_50'] = result_lvl_1.apply(lambda x: recall_at_k(x['als_rec_50'], x['actual'], 50), axis=1)
recall_total['own_rec_50'] = result_lvl_1.apply(lambda x: recall_at_k(x['own_rec_50'], x['actual'], 50), axis=1)
recall_total['sim_items_rec_50'] = result_lvl_1.apply(lambda x: recall_at_k(x['sim_items_rec_50'], x['actual'], 50), axis=1)
recall_total['sim_user_rec_50'] = result_lvl_1.apply(lambda x: recall_at_k(x['sim_user_rec_50'], x['actual'], 50), axis=1)

recall_total['als_rec_50'].mean(), recall_total['own_rec_50'].mean(), recall_total['sim_items_rec_50'].mean(), recall_total['sim_user_rec_50'].mean()

### Дают ли own recommendtions + top-popular лучший recall?

Да, лучший recall у recommendtions + top-popular (0,139)

In [ ]:
result_lvl_1['own_rec_20'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=20))
result_lvl_1['own_rec_50'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=50))
result_lvl_1['own_rec_100'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=100))
result_lvl_1['own_rec_500'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=500))


In [ ]:
recall_total['own_rec_20'] = result_lvl_1.apply(lambda x: recall_at_k(x['own_rec_20'], x['actual'], 20), axis=1)
recall_total['own_rec_50'] = result_lvl_1.apply(lambda x: recall_at_k(x['own_rec_50'], x['actual'], 50), axis=1)
recall_total['own_rec_100'] = result_lvl_1.apply(lambda x: recall_at_k(x['own_rec_100'], x['actual'], 100), axis=1)
recall_total['own_rec_500'] = result_lvl_1.apply(lambda x: recall_at_k(x['own_rec_500'], x['actual'], 500), axis=1)

In [ ]:
recall_total['own_rec_20'].mean(), recall_total['own_rec_50'].mean(), recall_total['own_rec_100'].mean(), recall_total['own_rec_200'].mean(), recall_total['own_rec_500'].mean()

B)* Как зависит recall@k от k? Постройте для одной схемы генерации кандидатов эту зависимость для k = {20, 50, 100, 200, 500}

C)* Исходя из прошлого вопроса, как вы думаете, какое значение k является наиболее разумным?

Чем, больше k, тем выше recall, но разумнее наверное брать наименьшее (k=20)

### Задание 2.

Обучите модель 2-ого уровня, при этом:

- Добавьте минимум по 2 фичи для юзера, товара и пары юзер-товар
- Измерьте отдельно precision@5 модели 1-ого уровня и двухуровневой модели на data_val_lvl_2
- Вырос ли precision@5 при использовании двухуровневой модели?

In [ ]:
users_lvl_2 = pd.DataFrame(data_train_lvl_2['user_id'].unique())
users_lvl_2.columns = ['user_id']
users_lvl_2['candidates'] = users_lvl_2['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=200))

In [ ]:
s = users_lvl_2.apply(lambda x: pd.Series(x['candidates']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'item_id'
users_lvl_2 = users_lvl_2.drop('candidates', axis=1).join(s)
users_lvl_2['drop'] = 1

users_lvl_2.head(2)

In [ ]:
targets_lvl_2 = data_train_lvl_2[['user_id', 'item_id']].copy()
targets_lvl_2['target'] = 1  # тут только покупки 

targets_lvl_2 = users_lvl_2.merge(targets_lvl_2, on=['user_id', 'item_id'], how='left')

targets_lvl_2['target'].fillna(0, inplace= True)

targets_lvl_2.drop_duplicates(subset =['user_id', 'item_id'], keep = 'first', inplace = True)

In [ ]:
data_train_lvl_2 = data_train_lvl_2.merge(item_features, on='item_id', how='left')
data_train_lvl_2 = data_train_lvl_2.merge(user_features, on='user_id', how='left')

data_train_lvl_2.head(2)

### Генерация фичей

In [ ]:
# Количество покупок по категориям
def create_feature_u_1(df=data_train_lvl_2):
    feat_u_1 = df[['user_id', 'department']].copy()
    feat_u_1 = pd.get_dummies(feat_u_1)
    feat_u_1 = feat_u_1.groupby(by='user_id').sum()

    return feat_u_1

In [ ]:
# Средний чек
def create_feature_u_2(df=data_train_lvl_2):
    feat_u_2 = df[['user_id', 'basket_id', 'sales_value']].copy()
    feat_u_2 = feat_u_2.groupby(by=['user_id', 'basket_id']).sum()
    feat_u_2 = feat_u_2.groupby(by=['user_id']).mean()
    feat_u_2.rename(columns={"sales_value": "average_bill"}, inplace=True)

    return feat_u_2

In [ ]:
# Кол-во покупок в неделю
def create_feature_i_1(df=data_train_lvl_2):
    feat_i_1 = df[['item_id', 'quantity', 'week_no']].copy()
    feat_i_1 = feat_i_1[feat_i_1['quantity'] > 0]
    feat_i_1 = feat_i_1.groupby(by=['item_id', 'week_no']).sum()
    feat_i_1 = feat_i_1.groupby(by=['item_id']).mean()
    feat_i_1.rename(columns={"quantity": "quantity_per_week"}, inplace=True)

    return feat_i_1

In [ ]:
# Средняя цена товара в категории
def create_feature_i_2(df=data_train_lvl_2):
    feat_i_2 = df[['item_id', 'quantity', 'sales_value', 'sub_commodity_desc']].copy()
    feat_i_2 = feat_i_2[feat_i_2['quantity'] > 0]
    feat_i_2['per_unit'] = feat_i_2['sales_value'] / feat_i_2['quantity']
    avr_per_comm = feat_i_2.groupby(by=['sub_commodity_desc']).mean()[['per_unit']].rename(columns={"per_unit": "avr_per_comm"})
    
    return avr_per_comm

In [ ]:
# (Средняя сумма покупки 1 товара в каждой категории (берем категорию item_id)) - (Цена item_id)
def create_feature_u_i_1(avr_per_comm, df=data_train_lvl_2):
    feat_u_i_1 = df[['user_id', 'item_id', 'quantity', 'sales_value', 'sub_commodity_desc']].copy()
    feat_u_i_1 = feat_u_i_1[feat_u_i_1['quantity'] > 0]
    feat_u_i_1 = feat_u_i_1.merge(avr_per_comm, on=['sub_commodity_desc'], how='left')  # avr_per_comm посчитан выше в feat_i_2
    feat_u_i_1['per_unit'] = feat_u_i_1['sales_value'] / feat_u_i_1['quantity']
    feat_u_i_1['diff_unit_avr'] = feat_u_i_1['per_unit'] - feat_u_i_1['avr_per_comm']
    feat_u_i_1 = feat_u_i_1[['user_id', 'item_id', 'diff_unit_avr']]

    return feat_u_i_1

In [ ]:
# (Кол-во покупок юзером конкретной категории в неделю) / (Среднее кол-во покупок всеми юзерами конкретной категории в неделю)
def create_feature_u_i_2(df=data_train_lvl_2):
    feat_u_i_2 = df[['user_id', 'quantity', 'sub_commodity_desc', 'week_no']].copy()
    feat_u_i_2 = feat_u_i_2[feat_u_i_2['quantity'] > 0]
    total_u = feat_u_i_2.groupby(by=['sub_commodity_desc', 'week_no']).sum()[['quantity']].rename(columns={"quantity": "quantity_per_week"})
    feat_u_i_2 = feat_u_i_2.groupby(by=['user_id', 'sub_commodity_desc', 'week_no'], as_index = False).sum()
    feat_u_i_2 = feat_u_i_2.merge(total_u, on=['sub_commodity_desc', 'week_no'], how='left')
    feat_u_i_2 = feat_u_i_2.groupby(by=['user_id', 'sub_commodity_desc'], as_index = False).mean()
    feat_u_i_2['diff_2'] = feat_u_i_2['quantity'] / feat_u_i_2['quantity_per_week']
    feat_u_i_2 = feat_u_i_2[['user_id', 'sub_commodity_desc', 'diff_2']]
    
    return feat_u_i_2

In [ ]:
# Сводная таблица
feat_u_1 = create_feature_u_1(df=data_train_lvl_2)
feat_u_2 = create_feature_u_2(df=data_train_lvl_2)
feat_i_1 = create_feature_i_1(df=data_train_lvl_2)
avr_per_comm = create_feature_i_2(df=data_train_lvl_2)
feat_u_i_1 = create_feature_u_i_1(avr_per_comm=avr_per_comm, df=data_train_lvl_2)
feat_u_i_2 = create_feature_u_i_2(df=data_train_lvl_2)


result = targets_lvl_2.merge(item_features[['item_id', 'sub_commodity_desc']], on=['item_id'], how='left')

result = result.merge(feat_u_1, on=['user_id'], how='left')
result = result.merge(feat_u_2, on=['user_id'], how='left')

result = result.merge(feat_i_1, on=['item_id'], how='left')
result = result.merge(avr_per_comm, on=['sub_commodity_desc'], how='left')

result = result.merge(feat_u_i_1, on=['user_id', 'item_id'], how='left')
result = result.merge(feat_u_i_2, on=['user_id', 'sub_commodity_desc'], how='left')

result.fillna(value=0, inplace=True)
result.head(2)

In [ ]:
le = LabelEncoder()
result["sub_commodity_desc"] = le.fit_transform(result["sub_commodity_desc"])

result.head(2)

### Train

In [ ]:
X_train = result.drop('target', axis=1)
y_train = result[['target']]

### Validation

In [ ]:
users_val = pd.DataFrame(data_val_lvl_2['user_id'].unique())
users_val.columns = ['user_id']
users_val['candidates'] = users_val['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=200))

s = users_val.apply(lambda x: pd.Series(x['candidates']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'item_id'
users_val = users_val.drop('candidates', axis=1).join(s)
users_val['drop'] = 1

targets_val = data_val_lvl_2[['user_id', 'item_id']].copy()
targets_val['target'] = 1  # тут только покупки 
targets_val = users_val.merge(targets_val, on=['user_id', 'item_id'], how='left')

targets_val['target'].fillna(0, inplace= True)
targets_val.drop_duplicates(subset =['user_id', 'item_id'], keep = 'first', inplace = True, ignore_index=True)

In [ ]:
# Добавляем фичи

result_val = targets_val.merge(item_features[['item_id', 'sub_commodity_desc']], on=['item_id'], how='left')

result_val = result_val.merge(feat_u_1, on=['user_id'], how='left')
result_val = result_val.merge(feat_u_2, on=['user_id'], how='left')

result_val = result_val.merge(feat_i_1, on=['item_id'], how='left')
result_val = result_val.merge(avr_per_comm, on=['sub_commodity_desc'], how='left')

result_val = result_val.merge(feat_u_i_1, on=['user_id', 'item_id'], how='left')
result_val = result_val.merge(feat_u_i_2, on=['user_id', 'sub_commodity_desc'], how='left')

result_val.fillna(value=0, inplace=True)
result_val.drop_duplicates(subset =['user_id', 'item_id'], keep = 'first', inplace = True, ignore_index=True)

In [ ]:
result_val["sub_commodity_desc"] = le.transform(result_val["sub_commodity_desc"])

In [ ]:
result_val = result_val.reindex(columns = result.columns, fill_value=0)

In [ ]:
X_val = result_val.drop('target', axis=1)
y_val = result_val[['target']]

### LightGBM

In [ ]:
%%time

lgb = LGBMClassifier(objective='binary', max_depth=7)
lgb.fit(X_train, y_train)

val_preds = lgb.predict_proba(X_val)

### Берем топ 5 для каждого юзера

In [ ]:
top_5 = targets_val.copy()
top_5['score'] = [x[1] for x in val_preds]
top_5 = top_5.sort_values(['user_id', 'score'], ascending=False).groupby('user_id').head(5)
top_5.head(5)

In [ ]:
# precision_at_5
top_5['target'].sum() / top_5.shape[0]

In [ ]:
# precision_at_5 у get_own_recommender
top_5_get_own = targets_val.groupby('user_id').head(5)
top_5_get_own['target'].sum() / top_5_get_own.shape[0]

In [ ]:
# тоже precision_at_5 у get_own_recommender для проверки
result_lvl_2 = data_val_lvl_2.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_2.columns=['user_id', 'actual']
result_lvl_2['own_rec_5'] = result_lvl_2['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=5))

result_lvl_2.apply(lambda x: precision_at_k(x['own_rec_5'], x['actual'], 5), axis=1).mean()